In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from helper_functions import create_tensorboard_callback
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.layers import Embedding
import random
import tensorflow_hub as hub
from helper_functions import calculate_results , plot_loss_curves

In [ ]:

train_df= pd.read_csv("../input/nlp-getting-started/train.csv")
test_df = pd.read_csv("../input/nlp-getting-started/test.csv")

In [ ]:
train_df.head() 

In [ ]:
test_df.head()

In [ ]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

In [ ]:
train_df.target.value_counts()

In [ ]:
test_id  =test_df["id"]
test_tweets = test_df["text"]
(test_id,test_tweets)

In [ ]:

random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index +5].itertuples():
  _,text,target = row
  print(f"Target :{target}" , "(real disaster)" if target > 0 else "(not a real disaster)")
  print(f"Text :\n{text}\n")
  print("-------------------------------------------------------\n")

**Splitting the train Data into train and validation datasets**

In [ ]:
train_tweets,val_tweets,train_target,val_target = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                  train_df_shuffled["target"].to_numpy(),
                                                                  test_size = 0.1,
                                                                  random_state =42)

In [ ]:
train_tweets[:10],train_target[:10]

In [ ]:
len(train_df_shuffled)

In [ ]:
len(train_tweets),len(train_target),len(val_tweets),len(val_target)

 ***Converting Text To Number***
 1. Text Vectorization
 2. Embedding

In [ ]:
max_vocab_length = 12000 # Total number of words in our vocabulary
max_length = 15  # round(sum([len(i.split()) for i in train_tweets])/len(train_tweets) ))  Sequence length to pad the outputs to.

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                   output_mode = "int",
                                   output_sequence_length = max_length)


In [ ]:
text_vectorizer.adapt(train_tweets)

In [ ]:
sample_sen = "Flying Raijin Jutsu"
print(f"Sample sentence  : \n{sample_sen}")
print(f"Vectorized sentence : \n{text_vectorizer(sample_sen)}")

In [ ]:
random_sen = random.choice(train_tweets)
print(f"random sentence : \n\n{random_sen}\
        Vectorized sentences : \n\n {text_vectorizer(random_sen)}")


In [ ]:
words_in_vocab = text_vectorizer.get_vocabulary()
words_in_vocab[:5],words_in_vocab[-5:] , len(words_in_vocab)

In [ ]:
embedding  = Embedding(input_dim = max_vocab_length,
                      output_dim =128 ,
                      input_length = max_length)

In [ ]:
random_tweet =random.choice(train_tweets)
print(f"Random tweet  : \n{random_tweet}\
        \n\n\nVectorized tweet : \n{text_vectorizer(random_tweet)}\
        \n\n\nEmbedded tweet :\n {embedding(text_vectorizer(random_tweet))}")

In [ ]:
DATA_DIR = "model_logs"

In [ ]:
import tensorflow_hub as hub

In [ ]:
embed_layer =hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",input_shape=[],
                            dtype="string",
                            trainable = False,
                            name= "USE")

In [ ]:
model_1 = tf.keras.Sequential([
    embed_layer,
    tf.keras.layers.Dense(512,activation="relu"),
    tf.keras.layers.Dense(1,activation = "sigmoid")
])

In [ ]:
model_1.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                optimizer=tf.keras.optimizers.SGD(lr=0.001 , momentum=0.9),
                metrics=["accuracy"])

In [ ]:
history_1 = model_1.fit(train_tweets,
            train_target,
            epochs=30,
            validation_data = (val_tweets,val_target))

In [ ]:
model_1_pred_probs=model_1.predict(val_tweets)
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_results = calculate_results(val_target, model_1_preds)
model_1_results

In [ ]:
plot_loss_curves(history_1)

In [ ]:
model_1_test_pred_probs = 

model_1_test_preds

In [ ]:
data=pd.DataFrame()
predicted = model_1.predict(test_tweets)
model_1_test_preds =  tf.squeeze(tf.round(model_1_test_pred_probs))
#model_1_test_preds = tf.reshape(model_1_test_preds, shape = (1,3263), name=None) 
data['id'] = test_id
data['target'] = model_1_test_preds

In [ ]:
data.to_csv('submission.csv',index = False)
data.head()